# 1. Устанавливаем и импортируем необходимые библиотеки

In [ ]:
!pip install ultralytics


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import shutil
import json
import tqdm
from ultralytics import YOLO

# 2. Подготавливаем данные для модели

In [ ]:
import os
import shutil
import json
import yaml
import random
from pathlib import Path

# пути к данным
SOURCE_IMAGES_DIR = '/kaggle/input/final-dataset/raw_data/images'
SOURCE_JSONS_DIR = '/kaggle/input/final-dataset/raw_data/jsons'
OUTPUT_BASE_DIR = '/kaggle/working/custom_data'

# очищаем выходные директории если они существуют
if os.path.exists(OUTPUT_BASE_DIR):
    shutil.rmtree(OUTPUT_BASE_DIR)

# создаем необходимые директории
for dir_path in [
    f'{OUTPUT_BASE_DIR}/images/train',
    f'{OUTPUT_BASE_DIR}/images/val',
    f'{OUTPUT_BASE_DIR}/labels/train',
    f'{OUTPUT_BASE_DIR}/labels/val'
]:
    os.makedirs(dir_path)

# словарь классов
classes = {
    'title': 0,
    'paragraph': 1,
    'table': 2,
    'picture': 3,
    'table_signature': 4,
    'picture_signature': 5,
    'numbered_list': 6,
    'marked_list': 7,
    'header': 8,
    'footer': 9,
    'footnote': 10,
    'formula': 11
}

def get_matching_files():
    """
    Получаем список файлов, у которых есть соответствующие пары json и png
    """
    image_files = {f.stem for f in Path(SOURCE_IMAGES_DIR).glob('*.png')}
    json_files = {f.stem for f in Path(SOURCE_JSONS_DIR).glob('*.json')}
    
    # находим пересечение множеств - файлы, у которых есть обе версии
    matching_files = sorted(list(image_files.intersection(json_files)))
    print(f"Total images found: {len(image_files)}")
    print(f"Total JSONs found: {len(json_files)}")
    print(f"Matching pairs found: {len(matching_files)}")
    return matching_files

def process_files(file_list, train_ratio=0.8):
    """
    Обрабатываем файлы, случайно распределяя их между train и val
    """
    # перемешиваем список файлов
    random.seed(42)  # для воспроизводимости
    random.shuffle(file_list)
    
    # определяем точку разделения
    split_point = int(len(file_list) * train_ratio)
    train_files = file_list[:split_point]
    val_files = file_list[split_point:]
    
    print(f"\nSplitting {len(file_list)} files:")
    print(f"Training files: {len(train_files)}")
    print(f"Validation files: {len(val_files)}")
    
    processed_train = 0
    processed_val = 0
    
    # обрабатываем тренировочные и валидационные файлы
    for is_train, files in [(True, train_files), (False, val_files)]:
        subset = 'train' if is_train else 'val'
        
        for filename in files:
            try:
                # копируем изображение
                src_img = os.path.join(SOURCE_IMAGES_DIR, f'{filename}.png')
                dst_img = os.path.join(OUTPUT_BASE_DIR, 'images', subset, f'{filename}.png')
                shutil.copy2(src_img, dst_img)
                
                # обрабатываем JSON и создаем YOLO-формат
                src_json = os.path.join(SOURCE_JSONS_DIR, f'{filename}.json')
                dst_txt = os.path.join(OUTPUT_BASE_DIR, 'labels', subset, f'{filename}.txt')
                
                with open(src_json) as f:
                    templates = json.loads(f.read())
                    
                with open(dst_txt, "w") as g:
                    for name in classes.keys():
                        if name in templates and len(templates[name]) > 0:
                            for obj in templates[name]:
                                # записываем в формате YOLO: class x_center y_center width height
                                g.write(f"{classes[name]} {((obj[2] + obj[0]) / 2) / templates['image_width']} "
                                      f"{((obj[3] + obj[1]) / 2) / templates['image_height']} "
                                      f"{(obj[2] - obj[0]) / templates['image_width']} "
                                      f"{(obj[3] - obj[1]) / templates['image_height']}\n")
                
                if is_train:
                    processed_train += 1
                else:
                    processed_val += 1
                    
            except Exception as e:
                print(f"Error processing file {filename}: {str(e)}")
                continue
    
    return processed_train, processed_val

def create_yaml_config():
    """
    Создаем конфигурационный YAML файл
    """
    data = {
        "path": OUTPUT_BASE_DIR,
        "train": 'images/train',
        "val": 'images/val',
        "names": {v: k for k, v in classes.items()}  
    }
    
    config_path = '/kaggle/working/config.yaml'
    with open(config_path, "w") as f:
        yaml.dump(data, f)
    return config_path

def main():
    # получаем список файлов
    matching_files = get_matching_files()
    
    # обрабатываем файлы
    processed_train, processed_val = process_files(matching_files)
    
    # создаем конфигурацию
    config_path = create_yaml_config()
    
    # выводим финальную статистику
    print(f"\nFinal statistics:")
    print(f"Successfully processed training images: {processed_train}")
    print(f"Successfully processed validation images: {processed_val}")
    print(f"Total processed: {processed_train + processed_val}")
    
    return config_path

# запускаем подготовку данных
config_path = main()

Ещё некоторые необходимые импорты

In [ ]:
!pip install -U ipywidgets
!pip install -U albumentations

# 3. Обучение модели YOLOv8 (предобученной)

In [ ]:
import torch
import os

# очистка памяти и настройка CUDA
torch.cuda.empty_cache()
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# загрузка модели
model = YOLO("/kaggle/input/yolov8_45epochs_trained/pytorch/default/1/best.pt")

# обучение
model.train(
    data=config_path,
    batch=32,
    epochs=15,
    lr0=1e-5,
    optimizer='AdamW',
    weight_decay=0.0001,
    augment=True,
    degrees=2.0,
    hsv_h=0.2,
    hsv_s=0.2,
    hsv_v=0.2,
    mosaic=0.0,
    flipud=0.0,
    fliplr=0.0,
    overlap_mask=False,
    freeze=10
)

# Результаты

Метрики находятся в одной папке с ноутбуком.